In [103]:
import pandas as pd

In [166]:
user = pd.read_csv('./data/user_artists_gp6.dat', delimiter='\t').sort_values('userID')
user[user['userID'] == 1893]

,userID,artistID,weight
24591,1893,17466,1
1971,1893,17465,1


In [105]:
artists = pd.read_csv('./data/artists_type.csv', delimiter=';', index_col=0)
artists.head()

,artist_id,first_music,similar_artists1,similar_artists2,similar_artists3,music_tag1,music_tag2,music_tag3,music_tag4,music_tag5
1,15353,Cara mia,0,0,0,pop,swedish,male vocalists,dance,melodifestivalen
2,16249,SAMBA DE 恋して,0,0,0,jazz,japanese,instrumental,j-jazz,samurai jazz
3,14476,You Can't Teach An Old Dog New Tricks,0,0,0,blues,blues rock,folk,singer-songwriter,country
4,13505,Yo era ninya,9064,0,0,jewish,world,sephardic,turkish,ladino
5,17630,Elektro Hexe,0,0,0,NaN,NaN,NaN,NaN,NaN


In [106]:
def get_fav_tag(userid):
    """
    This function takes an user id and return this favourite tags

    Arg:
        arg1: id of the user
    
    Return:
        A dictionary with as key each tag and as value their occurence
    """
    # We get all the rows that contain the userid and we keep only the artistID
    try:
        v_artists_listened = user[user['userID'] == int(userid)].sort_values('weight', ascending=False)['artistID'].to_list()   
        
        # We get only the rows of 'artsts' that are in 'v_reco' and drop all the columns except the music tags.
        artists_id = pd.DataFrame(artists[artists['artist_id'].isin(v_artists_listened)].drop(['first_music', 'similar_artists1', 'similar_artists2', 'similar_artists3'], axis=1))

        # We take the values of all the columns in a pandas Series
        tag_id = pd.Series(artists_id.iloc[:, 1:].values.flatten())

        # We return tag_id
        v_tags = tag_id.value_counts().head(5).to_dict()
        total = sum(v_tags.values())
        
        d_tags = {}
        for key, value in v_tags.items():
            d_tags[key] = round((value / total) * 100)
        return d_tags
    except:
        return dict({'0': 0, '0': 0, '0': 0, '0': 0, '0': 0})

In [115]:
user_tags = pd.DataFrame()
user_u = user['userID'].unique()

# Create a new DataFrame with 'userID' column
user_tags = pd.DataFrame({'userID': user_u})

# Convert 'userID' column to integer type
user_tags['userID'] = user_tags['userID'].astype(int)

print(user_tags)
print(type(user_tags['userID']))

      userID
0          2
1          3
2          4
3          5
4          6
...      ...
1881    2095
1882    2096
1883    2097
1884    2099
1885    2100

[1886 rows x 1 columns]
<class 'pandas.core.series.Series'>


In [116]:
get_fav_tag(2)

{'electronic': 32,
 'pop': 21,
 'female vocalists': 17,
 '80s': 17,
 'new wave': 13}

In [117]:
user_u

array([   2,    3,    4, ..., 2097, 2099, 2100])

In [118]:
user_tags['tags'] = user_tags['userID'].apply(lambda x: get_fav_tag(x))
user_tags.to_csv('./data/tags_by_usr.csv')

In [163]:
test = pd.read_csv('./data/tags_by_usr.csv', delimiter=',', index_col=0)
#test['tags'] = test['userID'].apply(lambda x: get_fav_tag(x))
test['tags'] = test['tags'].apply(lambda x: eval(x))
test.head()

,userID,tags
0,2,"{'electronic': 32, 'pop': 21, 'female vocalist..."
1,3,"{'ambient': 33, 'electronic': 25, 'experimenta..."
2,4,"{'rock': 31, 'alternative': 27, 'electronic': ..."
3,5,"{'alternative': 26, 'indie': 26, 'rock': 21, '..."
4,6,"{'female vocalists': 29, 'soul': 22, 'rnb': 18..."


In [164]:
print(test.tail())

      userID                                               tags
1881    2095  {'world': 28, 'folk': 28, 'turkish': 16, 'kurd...
1882    2096  {'alternative': 30, 'rock': 27, 'indie': 19, '...
1883    2097  {'alternative': 26, 'rock': 26, 'indie': 20, '...
1884    2099  {'electronic': 25, 'hip-hop': 22, 'trip-hop': ...
1885    2100  {'black metal': 44, 'metal': 20, 'french black...


In [131]:
def get_abs_dif_tags(new_tag, tag):
    tot = 0
    for i in tag:
        if i in new_tag:
            tot += abs(new_tag[i] - tag[i])
    if tot >= 100:
        return 100
    return 100 - tot


In [137]:
def find_reco_same_user(userid):
    tag = user_tags[user_tags.index == userid]['tags'].iloc[0]
    l_same_usr = {}
    equal = 5
    while (equal > 2 and len(l_same_usr) < 30):
        for i in range(len(user_tags)):
            if sum(1 for x, y in zip(list(user_tags['tags'][i].keys())[:equal], list(tag.keys())[:equal]) if x == y) == equal and user_tags.index[i] != userid:
                dif = {}
                l_same_usr[user_tags.index[i]] = get_abs_dif_tags(user_tags['tags'][i], tag)

            if (len(l_same_usr) >= 30):
                break
        equal = equal - 1
    if len(l_same_usr) < 30:
        for i in range(len(user_tags)):
            if sum(1 for x in list(user_tags['tags'][i].keys())[:3] for y in list(tag.keys())[:3] if x == y) == 3 and user_tags.index[i] != userid:
                dif = {}
                l_same_usr[user_tags.index[i]] = get_abs_dif_tags(user_tags['tags'][i], tag)
            if (len(l_same_usr) >= 30):
                break

    d_same_usr = dict(sorted(l_same_usr.items(), key=lambda x:x[1], reverse = True)[:10])
    d_usr = {key: value for key, value in d_same_usr.items() if value >= 80}
    ret = []
    for key in d_usr:
        ret.append(user[user.index == key].sort_values('weight', ascending=False)['artistID'].head(5).to_list())

    flattened_list = pd.Series([item for sublist in ret for item in sublist])
    fl = flattened_list.value_counts()
    return fl[fl.values > 1][:5]
    
    return dict(sorted(l_same_usr.items(), key=lambda x:x[1], reverse = True)[:5])

In [141]:
list_usr = user_tags['userID'].tolist()
for i in range(4, 5):
    if i in list_usr:
        tags = find_reco_same_user(int(i))
        print(tags)

{}
